## Taking Video as the input and extracting audio and text from it

In [2]:
import moviepy.editor as mp
import speech_recognition as sr

def extract_audio_from_video(video_path, audio_output_path):
    video_clip = mp.VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_output_path)
    audio_clip.close()
    video_clip.close()

if __name__ == "__main__":
    video_path = '1.mp4'
    audio_output_path = 'output.wav'
    extract_audio_from_video(video_path, audio_output_path)
    print("extracted audio")


OSError: MoviePy error: failed to read the duration of file "D:/STUDY/SEMESTER 7/NATURAL LANGUAGE PROCESSING/The World Through the Eyes of Animals.mp4".
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable

## Convert Audio to text

In [4]:
import os
import speech_recognition as sr
from pydub import AudioSegment

# Function to split audio into segments and transcribe each segment
def transcribe_audio_segments(audio_file_path, output_folder):
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    audio = AudioSegment.from_file(audio_file_path)

    # Duration of each segment in milliseconds (adjust as needed)
    segment_duration_ms = 5000  # 5 seconds

    # Calculate the number of segments
    num_segments = len(audio) // segment_duration_ms + 1

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # List to store segment transcriptions
    segment_transcriptions = []

    # Process and transcribe each segment
    for i in range(num_segments):
        start_time = i * segment_duration_ms
        end_time = (i + 1) * segment_duration_ms

        # Extract the segment
        segment = audio[start_time:end_time]

        # Transcribe the segment using Google Web Speech API
        try:
            # Export the segment as a temporary WAV file
            temp_audio_file = os.path.join(output_folder, f"segment_{i}.wav")
            segment.export(temp_audio_file, format="wav")

            # Recognize the segment
            with sr.AudioFile(temp_audio_file) as source:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)

            # Append the transcription to the list
            segment_transcriptions.append(text)

            print(f"Segment {i + 1} transcription saved")

        except sr.UnknownValueError:
            print(f"Segment {i + 1}: Google Web Speech API could not understand the audio")
        except sr.RequestError as e:
            print(f"Segment {i + 1}: Could not request results from Google Web Speech API; {e}")

    # Combine all segment transcriptions into one text file
    combined_text = "\n".join(segment_transcriptions)
    combined_output_file = os.path.join(output_folder, "combined_transcription.txt")

    with open(combined_output_file, 'w') as txt_file:
        txt_file.write(combined_text)

    print(f"Combined transcription saved to {combined_output_file}")

if __name__ == "__main__":
    input_audio_file = "output.wav"  # Replace with the path to your input audio file
    output_folder = "output_segments"  # Replace with the desired output folder

    transcribe_audio_segments(input_audio_file, output_folder)


Segment 1 transcription saved
Segment 2 transcription saved
Segment 3 transcription saved
Segment 4 transcription saved
Segment 5 transcription saved
Segment 6 transcription saved
Segment 7 transcription saved
Segment 8 transcription saved
Segment 9 transcription saved
Segment 10 transcription saved
Segment 11 transcription saved
Segment 12 transcription saved
Segment 13 transcription saved
Segment 14 transcription saved
Segment 15 transcription saved
Segment 16 transcription saved
Segment 17 transcription saved
Segment 18 transcription saved
Segment 19 transcription saved
Segment 20 transcription saved
Segment 21 transcription saved
Segment 22 transcription saved
Segment 23 transcription saved
Segment 24 transcription saved
Segment 25 transcription saved
Segment 26 transcription saved
Segment 27 transcription saved
Segment 28 transcription saved
Segment 29 transcription saved
Segment 30 transcription saved
Segment 31 transcription saved
Segment 32 transcription saved
Segment 33 transc

## Take extracted text as input and translate it to Traget language

In [5]:
#%pip install deep-translator

## text translation to gujarati

In [6]:
from deep_translator import GoogleTranslator
import re

# Function to split text into chunks with a maximum number of words
def split_text_by_words(text, max_words_per_chunk):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(' '.join(current_chunk)) + len(word) <= max_words_per_chunk:
            current_chunk.append(word)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Function to translate a list of text chunks and combine them
def translate_and_combine(input_text, source_language, target_language, max_words_per_chunk):
    try:
        # Split the input text into chunks by words
        text_chunks = split_text_by_words(input_text, max_words_per_chunk)

        # Initialize an empty list to store translated chunks
        translated_chunks = []

        # Translate each chunk and store the translations
        for chunk in text_chunks:
            translation = GoogleTranslator(source=source_language, target=target_language).translate(chunk)
            translated_chunks.append(translation)

        # Combine the translated chunks into a single string
        translated_text = ' '.join(translated_chunks)

        return translated_text
    except Exception as e:
        print("An error occurred:", str(e))

# Read input text from a file (replace 'input.txt' with your file path)
with open('output_segments/combined_transcription.txt', 'r', encoding='utf-8') as input_file:
    input_text = input_file.read()

source_language = 'auto'  # Auto-detect source language
target_language = 'gu'  # Gujarati
max_words_per_chunk = 5000  # Maximum words per chunk

# Remove extra spaces, tabs, and line breaks from the input text
input_text = ' '.join(input_text.split())

# Translate and combine the text
translated_text = translate_and_combine(input_text, source_language, target_language, max_words_per_chunk)

# Write the translated text to a file
with open('translated_text.txt', 'w', encoding='utf-8') as output_file:
    output_file.write(translated_text)

print("Translation saved to translated_text.txt")


Translation saved to translated_text.txt


In [7]:
#%pip install summarizer

In [8]:
#!pip install transformers

## Text Summarization

In [9]:
from transformers import pipeline
import textwrap

# Load the text summarization model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Read the input text from a file
with open('output_segments/combined_transcription.txt', 'r', encoding='utf-8') as file:
    input_text = file.read()

# Split the input text into smaller chunks
max_chunk_length = 1000  # Adjust this value as needed
chunks = textwrap.wrap(input_text, width=max_chunk_length)

# Initialize an empty summary
combined_summary = ""

# Summarize each chunk and append to the combined summary
for chunk in chunks:
    summary = summarizer(chunk, max_length=150, min_length=100, do_sample=False)[0]['summary_text']
    combined_summary += summary + "\n"

# Print the combined summarized text
print(combined_summary)

# Save the combined summarized text to an output file
with open('summary.txt', 'w', encoding='utf-8') as file:
    file.write(combined_summary)

'from transformers import pipeline\nimport textwrap\n\n# Load the text summarization model\nsummarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")\n\n# Read the input text from a file\nwith open(\'output_segments/combined_transcription.txt\', \'r\', encoding=\'utf-8\') as file:\n    input_text = file.read()\n\n# Split the input text into smaller chunks\nmax_chunk_length = 1000  # Adjust this value as needed\nchunks = textwrap.wrap(input_text, width=max_chunk_length)\n\n# Initialize an empty summary\ncombined_summary = ""\n\n# Summarize each chunk and append to the combined summary\nfor chunk in chunks:\n    summary = summarizer(chunk, max_length=150, min_length=100, do_sample=False)[0][\'summary_text\']\n    combined_summary += summary + "\n"\n\n# Print the combined summarized text\nprint(combined_summary)\n\n# Save the combined summarized text to an output file\nwith open(\'summary.txt\', \'w\', encoding=\'utf-8\') as file:\n    file.write(combined_summary)\n'

In [10]:
#%pip install gTTS wavio

In [13]:
from gtts import gTTS

# Function to convert text to audio and save as a WAV file
def text_to_audio(input_file_path, output_audio_file_path, language='gu'):
    try:
        # Read Gujarati text from the input .txt file
        with open(input_file_path, 'r', encoding='utf-8') as file:
            gujarati_text = file.read()

        # Initialize a gTTS object with the Gujarati text and language code
        tts = gTTS(text=gujarati_text, lang=language)

        # Save the generated audio as a WAV file
        tts.save(output_audio_file_path)

        print(f"Audio saved to {output_audio_file_path}")
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    # Define the path to your input .txt file containing Gujarati text
    input_file_path = "translated_text.txt"  

    # Define the path for saving the output audio as a WAV file
    output_audio_file_path = "output2.wav" 

    # Convert Gujarati text to audio and save as a WAV file
    text_to_audio(input_file_path, output_audio_file_path)


Audio saved to output2.wav


In [12]:
#%pip install Flask moviepy pydub SpeechRecognition deep_translator transformers pytube

In [15]:
%pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached pyttsx3-2.90-py3-none-any.whl (39 kB)
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/c2/a7/fe4bd49b5c4afa7a7ed3852abda6909e48c00715e6a134e47055381113aa/comtypes-1.2.0-py2.py3-none-any.whl.metadata
  Using cached pypiwin32-223-py3-none-any.whl (1.7 kB)
   ---------------------------------------- 184.3/184.3 kB 2.8 MB/s eta 0:00:00


In [20]:
%pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 981.5/981.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=c6f00272063a4b5bfd71ffba8754fdb0836a9da6f99ea6d598dd4a7123b1a777
  Stored in directory: c:\users\manvi\appdata\local\pip\cache\wheels\95\03\7d\59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [30]:
from gtts import gTTS

# Function to convert Gujarati text from a .txt file to audio
def text_to_audio(input_text_file, output_audio_file):
    try:
        # Read the Gujarati text from the input .txt file
        with open(input_text_file, 'r', encoding='utf-8') as file:
            input_text = file.read()

        # Create a gTTS (Google Text-to-Speech) object for Gujarati
        tts = gTTS(text=input_text, lang='gu')

        # Save the generated audio as a .mp3 file (or specify another format)
        tts.save(output_audio_file)

        print(f"Audio saved to {output_audio_file}")
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    # Define the path to your input .txt file containing Gujarati text
    input_text_file_path = "translated_text.txt"  # Replace with your input .txt file path

    # Define the path for saving the output audio file (in .mp3 format)
    output_audio_file_path = "output2.mp3"  # Replace with your desired output audio file path

    # Convert Gujarati text from the .txt file to audio
    text_to_audio(input_text_file_path, output_audio_file_path)


Audio saved to output2.mp3


In [31]:
import moviepy.editor as mp

# Function to synchronize dubbed audio with a video
def sync_audio_with_video(original_video_path, dubbed_audio_path, output_video_path):
    try:
        # Load the original video
        video_clip = mp.VideoFileClip(original_video_path)

        # Load the dubbed audio
        dubbed_audio_clip = mp.AudioFileClip(dubbed_audio_path)

        # Set the video's audio to the dubbed audio
        video_clip = video_clip.set_audio(dubbed_audio_clip)

        # Write the synchronized video with audio to the output file
        video_clip.write_videofile(output_video_path, codec='libx264', audio_codec='aac')

        print(f"Synchronized video saved to {output_video_path}")
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    # Define the paths to your original video, dubbed audio, and the output video
    original_video_path = "1.mp4"  # Replace with your original video file path
    dubbed_audio_path = "output2.mp3"  # Replace with your dubbed audio file path
    output_video_path = "output_video.mp4"  # Replace with your desired output video file path

    # Synchronize dubbed audio with the original video
    sync_audio_with_video(original_video_path, dubbed_audio_path, output_video_path)


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Synchronized video saved to output_video.mp4


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_file, flash